In [1]:

from re import A, L
import numpy as np
import pandas as pd
import argparse
from tqdm import tqdm
import torch
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import sys
import socket
import datetime
import random
import os
import matplotlib.pyplot as plt
from Bio import SeqIO
import pandas as pd
import numpy as np
import requests, sys
from util import *
from framepool import *
from popen import Auto_popen

tf.compat.v1.enable_eager_execution()

DATA = './../../data/utrdb2.csv'
BATCH_SIZE = 64
GENE = 'IFNG'
GC_LIMIT = 1.00
LR = 0.0001
GPU = '-1'
STEPS = 1000

if GPU == '-1':
    device = 'cpu'
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = GPU
    device = 'cuda'
    if GPU.includes(','):
        device = 'cuda:1'

DIM = 40
SEQ_LEN = 128
gpath = './../../models/checkpoint_3000.h5'
mrl_path = './../../models/utr_model_combined_residual_new.h5'
exp_path = './../../models/humanMedian_trainepoch.11-0.426.h5'
tpath = './script/checkpoint/RL_hard_share_MTL/3R/schedule_MTL-model_best_cv1.pth'

# exp_path = '/home/sina/UTR/models/GM12878_trainepoch.06-0.5062.h5'

CELL_LINE = ''
# CELL_LINE = 'K562_'
# CELL_LINE = 'GM12878_'



/Users/sbarazan/Documents/bilkent/utrgan/bioinformatics advances/code/UTRGAN/src/exp_optimization


In [2]:
def fetch_seq(start, end, chr, strand):
    server = "https://rest.ensembl.org"
    
    ext = "/sequence/region/human/" + str(chr) + ":" + str(start) + ".." + str(end) + ":" + str(strand) +  "?"
    
    r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})
    
    if not r.ok:
        r.raise_for_status()
        sys.exit()

    return r.text

def parse_biomart(path = 'martquery_0721120207_840.txt'):

    file = path

    fasta_sequences = SeqIO.parse(open(file),'fasta')

    genes = []
    ustarts = []
    uends = []
    seqs = []
    strands = []
    tsss = []
    chromosomes = []

    counter = 0

    for fasta in fasta_sequences:

        name, sequence = fasta.id, str(fasta.seq)
        
        if sequence != "Sequenceunavailable":

            counter += 1

            listed = name.split('|')
            # print(len(listed))

            if len(listed) == 8:

                genes.append(listed[1])
                chromosomes.append(listed[2])
                
                if ';' in listed[3]:
                    ustart = listed[3].split(';')[0]
                    uend = listed[4].split(';')[0]
                else:
                    ustart = listed[3]
                    uend = listed[4]

                strand = int(listed[0])

                if strand == -1:
                    ustarts.append(ustart)
                    uends.append(uend)
                else:
                    ustarts.append(uend)
                    uends.append(ustart)


                strands.append(str(strand))
                
                tsss.append(listed[-1])
                
                seqs.append(sequence)

    df = pd.DataFrame({'utr':seqs,'gene':genes, 'chr': chromosomes,'utr_start':ustarts,'utr_end':uends,'tss':tsss,'strand':strands})
    return df

def convert_model(model_:Model):
    input_ = tf.keras.layers.Input(shape=( 10500, 4))
    input = input_
    for i in range(len(model_.layers)-1):

        
        if isinstance(model_.layers[i+1],tf.keras.layers.Concatenate):
            paddings = tf.constant([[0,0],[0,6]])
            output = tf.pad(input, paddings, 'CONSTANT')
            input = output
        else:
            if not isinstance(model_.layers[i+1],tf.keras.layers.InputLayer):
                output = model_.layers[i+1](input)
                input = output

            if isinstance(model_.layers[i+1],tf.keras.layers.Conv1D):
                pass

    model = tf.keras.Model(inputs=input_, outputs=output)
    model.compile(loss="mse", optimizer="adam")
    return model

def one_hot(seq):
    convert = False
    if isinstance(seq, tf.Tensor):
        seq = seq.numpy().astype(str)
        convert = True

    num_seqs = len(seq)
    seq_len = len(seq[0])
    seqindex = {'A':0, 'C':1, 'G':2, 'T':3, 'a':0, 'c':1, 'g':2, 't':3}
    seq_vec = np.zeros((num_seqs,seq_len,4), dtype='bool')
    for i in range(num_seqs):
        thisseq = seq[i]
        for j in range(seq_len):
            try:
                seq_vec[i,j,seqindex[thisseq[j]]] = 1
            except:
                pass
    
    if convert:
        seq_vec = tf.convert_to_tensor(seq_vec,dtype=tf.float32)


    return seq_vec

def gen_random_dna(len=10500,size=SEQ_LEN):
    list_ = ['A','C','G','T']
    dnas = []
    for i in range(size):
            
        list_ = ['A','C','G','T']
        mydna = 'AGT'
        for i in range(len-3):
            char = list_[random.randint(0,3)]
            mydna = mydna + char
        dnas.append(mydna)


    return dnas
    
def select_dna_single(fname='small_seqs.npy',batch_size=64):
    refs = np.load(fname)
    indice = random.sample(range(0,refs.shape[0]),1)
    refs = refs
    return indice[0], refs    

def recover_seq(samples, rev_charmap):
    """Convert samples to strings and save to log directory."""
    if isinstance(samples,tf.Tensor):
        samples = samples.numpy()

    char_probs = samples
    argmax = np.argmax(char_probs, 2)
    seqs = []
    for line in argmax:
        s = "".join(rev_charmap[d] for d in line)
        s = s.replace('*','')
        seqs.append(s)

    seqs = np.array(seqs)
    return seqs

def replace_seqs_coordinate(original_utr_length, original_sequence, genutrs):
    replaced = []

    for i in range(len(genutrs)):
        utr = genutrs[i]
        utr_rep = original_sequence[:7000] + utr + original_sequence[7000+original_utr_length:]
        utr_rep = utr_rep[:10500]
        if len(utr_rep) < 10500:
            utr_rep = utr_rep + (10500 - len(utr_rep)) * 'A'

        replaced.append(utr_rep)

    return replaced

def replace_xpresso_seqs_single(gens,df:pd.DataFrame,refs,index):

    if isinstance(gens, tf.Tensor):
        gens = gens.numpy().astype('str')

    seqs = []
    originals = []
    len_ = abs(int(df.iloc[index]['utr_start']) - int(df.iloc[index]['utr_end']))

    origin_dna = refs[index]
    
    if len(origin_dna) != 10628:
        origin_dna = origin_dna + (10628-len(origin_dna)) * "A"
    

    for i in range(len(gens)):
        length = len(gens[i])
        diff = length - len_
        gen_dna = origin_dna[:7000] + gens[i] + origin_dna[7000+abs(int(df.iloc[index]['utr_start']) - int(df.iloc[index]['utr_end'])):10500 - diff]
        
        original = origin_dna[:10500]
        
        if len(gen_dna) < 10500:
            gen_dna = gen_dna + (10500 - len(gen_dna)) * "A"

        seqs.append(gen_dna)


    seqs = tf.convert_to_tensor(seqs)

    original = tf.convert_to_tensor([original for i in range(BATCH_SIZE)])

    return seqs, original

rna_vocab = {"A":0,
             "C":1,
             "G":2,
             "U":3,
             "*":4}

rev_rna_vocab = {v:k for k,v in rna_vocab.items()}

def select_best(scores, seqs, gc_control=False, GC=-1):
    t = np.max(scores,axis=1)
    # print(scores)
    maxinds = np.argmax(scores,axis=0)
    selected_scores = []
    selected_seqs = []
    for i in range(len(maxinds)):
        selected_seqs.append(seqs[maxinds[i]][i])
        selected_scores.append(scores[maxinds[i]][i])


    return selected_seqs, selected_scores

if __name__ == "__main__":

    model = load_model(exp_path)

    model = convert_model(model)

    gene_name = GENE

    if gene_name == 'TLR6':
        
        UTRSTARTS = [38856761,38829474,38843639]
        UTRENDS = [38856817,38829537,38843791]
        TSS = 38856817

        STRAND = -1
        CHR = 4

        UTRSTART = UTRSTARTS[0]
        UTREND = UTRENDS[0]

        # IFNG
        UTRSTART = 38856761
        UTREND = 38856817

        UTRLENGTH = UTREND - UTRSTART

    elif gene_name == 'IFNG':

        UTRSTARTS = [68159616]
        UTRENDS = [68159740]
        TSS = 68159740

        STRAND = -1
        CHR = 12

        UTRSTART = UTRSTARTS[0]
        UTREND = UTRENDS[0]

        UTRLENGTH = UTREND - UTRSTART

    elif gene_name == 'TNF':

        # TNF
        UTRSTARTS = [31575565]
        UTRENDS = [31575741]
        TSS = 31575565

        STRAND = 1
        CHR = 6

        UTRSTART = UTRSTARTS[0]
        UTREND = UTRENDS[0]

        UTRLENGTH = UTREND - UTRSTART

    elif gene_name == 'TP53':
        
        # TNF
        UTRSTARTS = [7687377,7676595]
        UTRENDS = [7687487,7676622]
        TSS = 7687487

        STRAND = -1
        CHR = 17

        UTRSTART = UTRSTARTS[0] 
        UTREND = UTRENDS[0]

        UTRLENGTH = abs(UTREND - UTRSTART)

    # original_gene_sequence = fetch_seq(start=TSS-7000,end=TSS+3500 + 128,chr=CHR,strand=STRAND)

    with open(f'./genes/{gene_name}.txt','r') as f:
        original_gene_sequence = f.readline()


    wgan = tf.keras.models.load_model(gpath)

    """
    Data:
    """

    noise = tf.Variable(tf.random.normal(shape=[BATCH_SIZE,DIM]))

    tf.random.set_seed(25)

    np.random.seed(25)

    diffs = []
    init_exps = []

    opt_exps = []

    orig_vals = []


    noise = tf.Variable(tf.random.normal(shape=[BATCH_SIZE,DIM]))
    # noise = tf.random.normal(shape=[BATCH_SIZE,40])
    noise_small = tf.random.normal(shape=[BATCH_SIZE,DIM],stddev=1e-5)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)

    '''
    Original Gene Expression
    '''

    seqs_orig = one_hot([original_gene_sequence[:10500]])
    pred_orig = model(seqs_orig) 


    '''
    Optimization takes place here.
    '''



    bind_scores_list = []
    bind_scores_means = []
    sequences_list = []



    iters_ = []

    OPTIMIZE = True

    DNA_SEL = False

    sequences_init = wgan(noise)

    gen_seqs_init = sequences_init.numpy().astype('float')

    seqs_gen_init = recover_seq(gen_seqs_init, rev_rna_vocab)

    seqs_init = replace_seqs_coordinate(UTRLENGTH, original_gene_sequence, seqs_gen_init)


    seqs_init = one_hot(seqs_init)

    pred_init = model(seqs_init) 

    t = tf.reshape(pred_init,(-1))

    init_t = t.numpy().astype('float')
    # sum_init = tf.reduce_max(t).numpy().astype('float')
    # sum_init = tf.math.argmax()

    mrl_model = load_framepool()
    te_model = torch.load(tpath,map_location=torch.device(device))['state_dict']  
    te_model.train().to(device)

    seqs_mrl = tf.convert_to_tensor(np.array([encode_seq_framepool(seq) for seq in seqs_gen_init]),dtype=tf.float32)
    seqs_te =  torch.transpose(torch.tensor(np.array(one_hot_all_motif(seqs_gen_init),dtype=np.float32)),2,1).float().to(device)

    mrl_preds_init = mrl_model(seqs_mrl).numpy().astype('float')
    te_preds_init = te_model.forward(seqs_te).cpu().data.numpy()

    means = []
    maxes = []
    
    STEPS = STEPS

    seqs_collection = []
    scores_collection = []

    GC_CONTROL = False
    GC_Limit = 0.65

    if GC_LIMIT > 0.:
        GC_CONTROL = True

    if OPTIMIZE:
        
        iter_ = 0
        for opt_iter in tqdm(range(STEPS)):
            
            with tf.GradientTape() as gtape:

                gtape.watch(noise)
                
                sequences = wgan(noise)

                seqs_gen = recover_seq(sequences, rev_rna_vocab)
                seqs_collection.append(seqs_gen)

                seqs2 = replace_seqs_coordinate(UTRLENGTH, original_gene_sequence, seqs_gen)
            
                seqs = one_hot(seqs2)
                seqs = tf.convert_to_tensor(seqs,dtype=tf.float32)


                with tf.GradientTape() as ptape:

                    ptape.watch(seqs)

                    pred =  model(seqs)
                    t = tf.reshape(pred,(-1))
                    scores_collection.append(t.numpy().astype('float'))
                    mx = np.amax(t.numpy().astype('float'),axis=0)
                    mx = np.max(mx)
                    
                    sum_ = tf.reduce_sum(t)
                    maxes.append(mx)
                    means.append(sum_/BATCH_SIZE)
                    pred = tf.math.scalar_mul(-1.0, pred)

                g1 = ptape.gradient(pred,seqs)

                g1 = tf.slice(g1,[0,7000,0],[-1,SEQ_LEN,-1])


                tmp_g = g1.numpy().astype('float')
                tmp_seqs = seqs_gen

                tmp_lst = np.zeros(shape=(BATCH_SIZE,SEQ_LEN,5))
                for i in range(len(tmp_seqs)):
                    len_ = len(tmp_seqs[i])
                    
                    edited_g = tmp_g[i][:len_,:]

                    edited_g = np.pad(edited_g,((0,SEQ_LEN-len_),(0,1)),'constant')   
                    
                    tmp_lst[i] = edited_g
                    
                g1 = tf.convert_to_tensor(tmp_lst,dtype=tf.float32)

                g2 = gtape.gradient(sequences,noise,output_gradients=g1)

            a1 = g2 + noise_small
            change = [(a1,noise)]
            
            optimizer.apply_gradients(change)

            iters_.append(iter_)
            iter_ += 1

        sequences_opt = wgan(noise)

        gen_seqs_opt = sequences_opt.numpy().astype('float')

        seqs_gen_opt = recover_seq(gen_seqs_opt, rev_rna_vocab)

        seqs_opt= replace_seqs_coordinate(UTRLENGTH, original_gene_sequence, seqs_gen_opt)

        seqs_opt = one_hot(seqs_opt)

        pred_opt = model(seqs_opt)

        t = tf.reshape(pred_opt,(-1))
        opt_t = t.numpy().astype('float')

        # print(seqs_collection)
        # print(scores_collection)

        if GC_CONTROL:
            best_seqs, best_scores = select_best(scores_collection, seqs_collection, True, GC_Limit)
        else:
            best_seqs, best_scores = select_best(scores_collection, seqs_collection)


        seqs_mrl = tf.convert_to_tensor(np.array([encode_seq_framepool(seq) for seq in seqs_gen_opt]),dtype=tf.float32)
        seqs_te =  torch.transpose(torch.tensor(np.array(one_hot_all_motif(seqs_gen_opt),dtype=np.float32)),2,1).float().to(device)

        mrl_preds_opt = mrl_model(seqs_mrl).numpy().astype('float')
        te_preds_opt = te_model.forward(seqs_te).cpu().data.numpy()

        if GC_CONTROL:

            with open(f'./outputs/{CELL_LINE}gc_init_exps_'+gene_name+'.txt', 'w') as f:
                for item in init_t:
                    f.write(f'{item}\n')

            with open(f'./outputs/{CELL_LINE}gc_opt_exps_'+gene_name+'.txt', 'w') as f:
                for item in best_scores:
                    f.write(f'{item}\n')

            with open(f'./outputs/{CELL_LINE}gc_best_seqs_'+gene_name+'.txt', 'w') as f:
                for item in best_seqs:
                    f.write(f'{item}\n')

            with open(f'./outputs/{CELL_LINE}gc_init_seqs_'+gene_name+'.txt', 'w') as f:
                for item in seqs_gen_init:
                    f.write(f'{item}\n')

        else:
            with open(f'./outputs/{CELL_LINE}init_exps_{gene_name}.txt', 'w') as f:
                for item in init_t:
                    f.write(f'{item}\n')

            with open(f'./outputs/{CELL_LINE}opt_exps_{gene_name}.txt', 'w') as f:
                for item in best_scores:
                    f.write(f'{item}\n')

            with open(f'./outputs/{CELL_LINE}best_seqs_{gene_name}.txt', 'w') as f:
                for item in best_seqs:
                    f.write(f'{item}\n')

            with open(f'./outputs/{CELL_LINE}init_seqs_{gene_name}.txt', 'w') as f:
                for item in seqs_gen_init:
                    f.write(f'{item}\n')


        print(f"Average Initial Expression: {np.average(init_t)}")
        print(f"Best Expression: {np.average(best_scores)}")


/opt/anaconda3/envs/utrgan/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
100%|██████████| 1000/1000 [04:15<00:00,  3.91it/s]


FileNotFoundError: [Errno 2] No such file or directory: './outputs/gc_init_exps_IFNG.txt'

In [6]:
import requests

def get_gene_genomic_info(gene_name, species="homo_sapiens"):
    """
    Given a gene symbol, retrieves gene-level data including:
      - Ensembl gene ID,
      - Gene start and end coordinates,
      - Strand and computed TSS,
      - Genomic sequence of the gene,
      - UTR segments overlapping the gene region (genomic coordinates).
    """
    base_url = "https://rest.ensembl.org"
    headers = {"Content-Type": "application/json"}

    # Step 1: Get gene ID using the gene symbol
    xref_url = f"{base_url}/xrefs/symbol/{species}/{gene_name}?object_type=gene"
    response = requests.get(xref_url, headers=headers)
    if not response.ok:
        raise Exception(f"Error fetching gene ID for symbol {gene_name}: {response.status_code}")
    xref_data = response.json()
    if not xref_data:
        raise Exception(f"No gene found for symbol '{gene_name}'.")
    gene_id = xref_data[0]['id']

    # Step 2: Retrieve gene-level details (without transcript expansion)
    lookup_url = f"{base_url}/lookup/id/{gene_id}?expand=0"
    response = requests.get(lookup_url, headers=headers)
    if not response.ok:
        raise Exception(f"Error fetching details for gene {gene_id}: {response.status_code}")
    gene_data = response.json()

    gene_start = gene_data.get("start")
    gene_end = gene_data.get("end")
    strand = gene_data.get("strand")
    seq_region_name = gene_data.get("seq_region_name")
    # Determine TSS: gene start if on the plus strand, gene end if on the minus strand
    tss = gene_start if strand == 1 else gene_end

    # Step 3: Retrieve the gene's genomic sequence
    seq_url = f"{base_url}/sequence/id/{gene_id}?type=genomic"
    response = requests.get(seq_url, headers=headers)
    gene_sequence = response.json().get("seq") if response.ok else None

    # Step 4: Retrieve UTR features from the gene region using the overlap/region endpoint.
    # Note: This returns UTR features from all transcripts overlapping the gene region.
    region = f"{seq_region_name}:{gene_start}-{gene_end}"
    utr_url = f"{base_url}/overlap/region/{species}/{region}?feature=utr"
    response = requests.get(utr_url, headers=headers)
    utr_info = response.json() if response.ok else []
    
    result = {
        "gene_id": gene_id,
        "gene_name": gene_name,
        "gene_start": gene_start,
        "gene_end": gene_end,
        "strand": strand,
        "TSS": tss,
        "gene_sequence": gene_sequence,
        "UTRs": utr_info
    }

    return result

if __name__ == "__main__":
    gene_name = input("Enter gene name: ").strip()
    try:
        info = get_gene_genomic_info(gene_name)
    except Exception as e:
        print(e)
    else:
        print(f"Gene ID: {info['gene_id']}")
        print(f"Gene Start: {info['gene_start']}")
        print(f"Gene End: {info['gene_end']}")
        print(f"Strand: {info['strand']}")
        print(f"TSS: {info['TSS']}")
        if info['gene_sequence']:
            print(f"Gene Sequence (first 100 bases): {info['gene_sequence'][:100]} ...")
        else:
            print("Gene sequence not available.")
        
        if info['UTRs']:
            print("UTR segments (genomic coordinates):")
            for utr in info['UTRs']:
                print(f"  - Chromosome: {utr.get('seq_region_name')}, Start: {utr.get('start')}, End: {utr.get('end')}")
        else:
            print("No UTR information found in the gene region.")



Gene ID: ENSG00000111537
Gene Start: 68154768
Gene End: 68159740
Strand: -1
TSS: 68159740
Gene Sequence (first 100 bases): ACATTGTTCTGATCATCTGAAGATCAGCTATTAGAAGAGAAAGATCAGTTAAGTCCTTTGGACCTGATCAGCTTGATACAAGAACTACTGATTTCAACTT ...
No UTR information found in the gene region.


In [12]:
import requests
import json
import time

class GeneInfoRetriever:
    def __init__(self):
        self.base_url = "https://rest.ensembl.org"
        self.headers = {"Content-Type": "application/json"}
        self.sleep_time = 0.5  # To respect Ensembl API rate limits

    def get_gene_id(self, gene_symbol, species="human"):
        """Retrieve the Ensembl gene ID for a gene symbol."""
        species_map = {"human": "homo_sapiens", "mouse": "mus_musculus"}
        species_name = species_map.get(species.lower(), species)
        
        endpoint = f"/lookup/symbol/{species_name}/{gene_symbol}"
        response = self._make_request(endpoint)
        
        if response:
            return response.get("id")
        return None

    def get_gene_sequence(self, gene_id):
        """Retrieve the gene sequence for a gene ID."""
        endpoint = f"/sequence/id/{gene_id}?type=genomic"
        response = self._make_request(endpoint)
        
        if response:
            return response.get("seq")
        return None

    def get_gene_coordinates(self, gene_id):
        """Retrieve the genomic coordinates for a gene ID."""
        endpoint = f"/lookup/id/{gene_id}?expand=1"
        response = self._make_request(endpoint)
        
        if response:
            return {
                "chromosome": response.get("seq_region_name"),
                "start": response.get("start"),
                "end": response.get("end"),
                "strand": response.get("strand")
            }
        return None

    def get_tss_location(self, gene_id):
        """
        Retrieve the transcription start site (TSS) location.
        The TSS is typically at the start of the gene on the positive strand,
        or at the end of the gene on the negative strand.
        """
        coords = self.get_gene_coordinates(gene_id)
        
        if coords:
            # For genes on the positive strand, TSS is at the start
            # For genes on the negative strand, TSS is at the end
            tss = coords["start"] if coords["strand"] == 1 else coords["end"]
            return {
                "chromosome": coords["chromosome"],
                "position": tss,
                "strand": coords["strand"]
            }
        return None

    def get_transcript_info(self, gene_id):
        """Get information about all transcripts for a gene, including UTRs."""
        # This endpoint needs to be expanded to include UTRs
        endpoint = f"/lookup/id/{gene_id}?expand=1&utr=1"
        response = self._make_request(endpoint)
        
        transcripts = []
        if response and "Transcript" in response:
            for transcript in response["Transcript"]:
                transcript_info = {
                    "transcript_id": transcript.get("id"),
                    "start": transcript.get("start"),
                    "end": transcript.get("end"),
                    "strand": transcript.get("strand"),
                    "UTRs": []
                }
                
                # Extract UTR information based on actual Ensembl API response structure
                if "UTR" in transcript:
                    for utr in transcript["UTR"]:
                        utr_type = "5' UTR" if utr.get("object_type") == "five_prime_UTR" else "3' UTR"
                        transcript_info["UTRs"].append({
                            "start": utr.get("start"),
                            "end": utr.get("end"),
                            "type": utr_type
                        })
                
                transcripts.append(transcript_info)
                
        return transcripts

    def _make_request(self, endpoint):
        """Make a request to the Ensembl REST API."""
        url = self.base_url + endpoint
        
        try:
            response = requests.get(url, headers=self.headers)
            time.sleep(self.sleep_time)  # Be nice to the API
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error: {response.status_code} - {response.text}")
                return None
        except Exception as e:
            print(f"Request error: {e}")
            return None

    def get_all_gene_info(self, gene_symbol, species="human"):
        """Retrieve all information about a gene."""
        # Get the Ensembl gene ID
        gene_id = self.get_gene_id(gene_symbol, species)
        if not gene_id:
            return {"error": f"Gene {gene_symbol} not found"}
        
        # Get the gene sequence
        sequence = self.get_gene_sequence(gene_id)
        
        # Get the gene coordinates
        coordinates = self.get_gene_coordinates(gene_id)
        
        # Get the TSS location
        tss = self.get_tss_location(gene_id)
        
        # Get transcript information (including UTRs)
        transcripts = self.get_transcript_info(gene_id)
        
        return {
            "gene_symbol": gene_symbol,
            "gene_id": gene_id,
            "sequence": sequence,
            "coordinates": coordinates,
            "tss": tss,
            "transcripts": transcripts
        }

def main():
    # Predefined variables - change these values as needed
    gene_symbol = "BRCA1"  # Example gene
    species = "human"      # Default species
    
    retriever = GeneInfoRetriever()
    result = retriever.get_all_gene_info(gene_symbol, species)
    
    # Print results
    if "error" in result:
        print(result["error"])
    else:
        print(f"Gene: {result['gene_symbol']} (Ensembl ID: {result['gene_id']})")
        print("\nGenome Coordinates:")
        print(f"Chromosome: {result['coordinates']['chromosome']}")
        print(f"Start: {result['coordinates']['start']}")
        print(f"End: {result['coordinates']['end']}")
        print(f"Strand: {'+' if result['coordinates']['strand'] == 1 else '-'}")
        
        print("\nTranscription Start Site (TSS):")
        print(f"Chromosome: {result['tss']['chromosome']}")
        print(f"Position: {result['tss']['position']}")
        
        print("\nTranscripts:")
        for i, transcript in enumerate(result['transcripts'], 1):
            print(f"\nTranscript {i}: {transcript['transcript_id']}")
            print(f"Start: {transcript['start']}")
            print(f"End: {transcript['end']}")
            
            if transcript['UTRs']:
                print("UTRs:")
                for utr in transcript['UTRs']:
                    print(f"  {utr['type']}: {utr['start']}-{utr['end']}")
        
        print("\nSequence (first 100 bp):")
        if result['sequence']:
            print(result['sequence'][:100] + "...")
            print(f"Total sequence length: {len(result['sequence'])} bp")

if __name__ == "__main__":
    main()

Gene: BRCA1 (Ensembl ID: ENSG00000012048)

Genome Coordinates:
Chromosome: 17
Start: 43044295
End: 43170245
Strand: -

Transcription Start Site (TSS):
Chromosome: 17
Position: 43170245

Transcripts:

Transcript 1: ENST00000497488
Start: 43044295
End: 43125300
UTRs:
  5' UTR: 43125271-43125300
  5' UTR: 43094643-43094860
  3' UTR: 43044295-43045677

Transcript 2: ENST00000489037
Start: 43044295
End: 43125321
UTRs:
  5' UTR: 43125182-43125321
  5' UTR: 43124097-43124115
  3' UTR: 43044295-43045677

Transcript 3: ENST00000478531
Start: 43044295
End: 43125359
UTRs:
  5' UTR: 43125277-43125359
  5' UTR: 43124097-43124115
  3' UTR: 43044295-43045677

Transcript 4: ENST00000357654
Start: 43044295
End: 43125364
UTRs:
  5' UTR: 43125271-43125364
  5' UTR: 43124097-43124115
  3' UTR: 43044295-43045677

Transcript 5: ENST00000473961
Start: 43044295
End: 43125364
UTRs:
  5' UTR: 43125271-43125364
  5' UTR: 43124097-43124115
  3' UTR: 43044295-43045677

Transcript 6: ENST00000477152
Start: 43044295